# Capstone Report

## introduction/Business Problem

In this project, we will be seeing if Forsquare data can be a good predictor of any demographic data. This will help citizens and governments use data to understand neighborhoods when data is not already sufficiently available.

## The data

This project uses two main sources of data. The first is from Foursquare, a local search-and-discovery app which provides personalized recommendations of places to go near a specific location. To get this information we used one of Foursauqre's API. We will then make calls to the API using a list of Toronto neighborhoods.

The second data source contains demographic information on Toronto's neighborhoods. This data comes from the city of Toronto's Open Data Portal (https://portal0.cf.opendata.inter.sandbox-toronto.ca/).

### Foursquare Data

In [53]:
projectID = "***"
projectToken = "***"

clientID = "***"
clientSecret = "***"

import json, requests
from project_lib import Project
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import matplotlib.pyplot as plt
import json, requests

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id='WXOU4YCYUPFD0MWX4OVS1XZRJ4X1BL01KGGPXTRPQYFO5MEO',
  client_secret='3ST5S3BQFBULNRR3OXZ3RGRT12CXJG0ZJ3N5KTKESEPGYULC',
  v='20180323',
  ll='43.8066863,-79.1943534',
  limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [24]:
print('Name:', data['response']['groups'][0]['items'][0]['venue']['name'])
print('Location:', data['response']['groups'][0]['items'][0]['venue']['location']['city'], data['response']['groups'][0]['items'][0]['venue']['location']['state'])

Name: African Rainforest Pavilion
Location: Toronto ON


### Demographic Data

In [25]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv', index_col = 3, encoding = 'iso-8859-1')
df = df.drop(['Category', 'Topic', 'Data Source'], axis = 1).T
df.head()

Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Internal migrants,Intraprovincial migrants,Interprovincial migrants,External migrants
City of Toronto,NaN,NaN,"2,731,571","2,615,060",4.50%,"1,179,057","1,112,929","4,334",630.2,"398,135",...,"59,945","2,556,120","1,516,110","1,040,015","639,060","400,950","184,120","141,135","42,985","216,835"
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,605,"27,490","18,865","8,610","5,445","3,170",880,735,135,"2,280"
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,490,"22,325","13,565","8,775","5,610","3,145",980,760,220,"2,170"
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,70,"11,370","8,235","3,130","2,200",925,680,615,70,245
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,835,"27,715","12,980","14,735","8,340","6,390","3,930","2,630","1,310","2,460"


In [26]:

neighborhoods = df.index.tolist()
neighborhoods[0:5]

['City of Toronto',
 'Agincourt North',
 'Agincourt South-Malvern West',
 'Alderwood',
 'Annex']

## The Methadology

### About the data

Foursquare Data

This data is accessible through the company's API.
After collecting recommended venues for each neighborhood, we made Details calls for each venue in the search results to get a set of details about a venue including location, tips, and categories. Because the Details call is a premium call, we were only able to make 500 calls a day. As such, we decided to limit the number of neighborhoods included in our analysis and search 50 venues (the max limit per Explore call) for each neighborhood. The list of neighborhoods was determined randomly.

Demographic Data 

Demographic data on Toronto neighborhoods was collected from the City of Toronto's Open Data Portal. The csv file was read directly from the download link and stored into a dataframe. To make the data usable for our purposes,unnecessary data has been removed.

In [27]:
demo_df = pd.read_csv('https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv', index_col = 3, encoding = 'iso-8859-1')
demo_df = demo_df.drop(['Category', 'Topic', 'Data Source'], axis = 1).T
demo_df.head()

Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Internal migrants,Intraprovincial migrants,Interprovincial migrants,External migrants
City of Toronto,NaN,NaN,"2,731,571","2,615,060",4.50%,"1,179,057","1,112,929","4,334",630.2,"398,135",...,"59,945","2,556,120","1,516,110","1,040,015","639,060","400,950","184,120","141,135","42,985","216,835"
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,605,"27,490","18,865","8,610","5,445","3,170",880,735,135,"2,280"
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,490,"22,325","13,565","8,775","5,610","3,145",980,760,220,"2,170"
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,70,"11,370","8,235","3,130","2,200",925,680,615,70,245
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,835,"27,715","12,980","14,735","8,340","6,390","3,930","2,630","1,310","2,460"


In [28]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id='WXOU4YCYUPFD0MWX4OVS1XZRJ4X1BL01KGGPXTRPQYFO5MEO',
  client_secret='3ST5S3BQFBULNRR3OXZ3RGRT12CXJG0ZJ3N5KTKESEPGYULC',
  v='20180323',
  ll='43.8066863,-79.1943534',
  limit=50
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [29]:
print('Name:', data['response']['groups'][0]['items'][0]['venue']['name'])
print('Location:', data['response']['groups'][0]['items'][0]['venue']['location']['city'], data['response']['groups'][0]['items'][0]['venue']['location']['state'])

Name: African Rainforest Pavilion
Location: Toronto ON


In [31]:
neighborhoods = pd.DataFrame(data = {'Neighborhood':['Agincourt North', 'Alderwood', 'Annex', 'Bathurst Manor', 'Bayview Village', 
                                                'Cliffcrest', 'Dorset Park', 'Flemingdon Park', 'Forest Hill North', 'Guildwood', 
                                                'Henry Farm', 'Highland Creek', 'Hillcrest Village', 'Humber Summit', 'Ionview', 
                                                'Kennedy Park', 'Little Portugal', 'Long Branch', 'Malvern', 'Markland Wood',
                                                'Morningside', 'Mount Dennis', 'New Toronto', 'Oakridge', 'Regent Park', 
                                                'Roncesvalles', 'Rouge', 'Scarborough Village', 'The Beaches', 'Thorncliffe Park', 
                                                'West Hill', 'Weston', 'Woburn'], 
                               'LL':['43.807553, -79.268030', '43.604498, -79.541228', '43.671220, -79.404673', 
                                     '43.765731, -79.457152', '43.777287, -79.378287', '43.725333, -79.231912', 
                                     '43.759675, -79.278793', '43.716436, -79.331415', '43.704375, -79.427817', 
                                     '43.748745, -79.194773', '43.771804, -79.341777', '43.792310, -79.176724', 
                                     '43.803083, -79.353281', '43.759372, -79.555372', '43.736031, -79.272930', 
                                     '43.726305, -79.260049', '43.648403, -79.430358', '43.592414, -79.533756', 
                                     '43.806878, -79.218038', '43.634009, -79.575008', '43.782131, -79.203709', 
                                     '43.688643, -79.498612', '43.601285, -79.510259', '43.696451, -79.280519', 
                                     '43.659917, -79.361629', '43.647618, -79.449624', '43.804952, -79.165872', 
                                     '43.740604, -79.215696', '43.671681, -79.298558', '43.708681, -79.348812', 
                                     '43.769107, -79.176750', '43.703432, -79.517863', '43.768958, -79.227792']})

# merge the neighborhood locations and demographic data
neighborhoods = neighborhoods.merge(demo_df, how='left', left_on='Neighborhood', right_index = True)
neighborhoods.shape

(33, 2385)

In [12]:

neighborhoods.head()

,Neighborhood,LL,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,...,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Internal migrants,Intraprovincial migrants,Interprovincial migrants,External migrants
0,Agincourt North,"43.807553, -79.268030",129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",...,605,"27,490","18,865","8,610","5,445","3,170",880,735,135,"2,280"
1,Alderwood,"43.604498, -79.541228",20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",...,70,"11,370","8,235","3,130","2,200",925,680,615,70,245
2,Annex,"43.671220, -79.404673",95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",...,835,"27,715","12,980","14,735","8,340","6,390","3,930","2,630","1,310","2,460"
3,Bathurst Manor,"43.765731, -79.457152",34,No Designation,"15,873","15,434",2.80%,"6,418","6,089","3,377",...,375,"14,750","8,845","5,905","3,680","2,235",915,745,170,"1,310"
4,Bayview Village,"43.777287, -79.378287",52,No Designation,"21,396","17,671",21.10%,"10,111","9,532","4,195",...,625,"20,250","9,055","11,200","6,175","5,015","1,995","1,485",510,"3,030"


In [56]:
df = pd.DataFrame(columns=['id', 'name', 'lat','lng','neighborhood'])

url = 'https://api.foursquare.com/v2/venues/explore'

for i in range(0,len(neighborhoods.index)):

    params = dict(
        client_id=clientID,
        client_secret=clientSecret,
        v='20180323',
        section='food',
        time='any',
        day='any',
        ll=neighborhoods.LL[i],
        sortByDistance=1,
        limit=50
    )
    
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    
    for j in range(0,50):
        ID = data['response']['groups'][0]['items'][j]['venue']['id']
        name = data['response']['groups'][0]['items'][j]['venue']['name']
        lat = data['response']['groups'][0]['items'][j]['venue']['location']['lat']
        lng = data['response']['groups'][0]['items'][j]['venue']['location']['lng']
        df = df.append(pd.DataFrame([[ID, name, lat, lng, neighborhoods.Neighborhood[i]]], columns=['id', 'name', 'lat','lng','neighborhood']), ignore_index = True)

KeyError: 'groups'

In [57]:
df.shape

(0, 5)

In [44]:
neighborhoods_short = pd.concat([neighborhoods['LL'], 
                                 neighborhoods['Neighborhood'],
                                 neighborhoods['Population density per square kilometre'].str.replace(',', '').astype(int),
                                 neighborhoods['Population Change 2011-2016'].str.replace('%', '').astype(float)], 
                                axis = 1)

temp = pd.concat([neighborhoods['Children (0-14 years)'].str.replace(',', '').astype(int), 
                 neighborhoods['Youth (15-24 years)'].str.replace(',', '').astype(int), 
                 neighborhoods['Working Age (25-54 years)'].str.replace(',', '').astype(int), 
                 neighborhoods['Pre-retirement (55-64 years)'].str.replace(',', '').astype(int), 
                 neighborhoods['Seniors (65+ years)'].str.replace(',', '').astype(int), 
                 neighborhoods['Older Seniors (85+ years)'].str.replace(',', '').astype(int)], axis = 1)
total = temp.iloc[:,0] + temp.iloc[:,1] + temp.iloc[:,2] + temp.iloc[:,3] + temp.iloc[:,4] + temp.iloc[:,5]
temp.iloc[:,0] = temp.iloc[:,0] / total
temp.iloc[:,1] = temp.iloc[:,1] / total
temp.iloc[:,2] = temp.iloc[:,2] / total
temp.iloc[:,3] = temp.iloc[:,3] / total
temp.iloc[:,4] = temp.iloc[:,4] / total
temp.iloc[:,5] = temp.iloc[:,5] / total
neighborhoods_short = pd.concat([neighborhoods_short, temp], axis = 1)

temp = pd.concat([neighborhoods['  Married or living common law'].astype(int),
                  neighborhoods['  Not married and not living common law'].astype(int)], axis = 1) 
total = temp.iloc[:,0] + temp.iloc[:,1]
temp.iloc[:,0] = temp.iloc[:,0] / total
temp.iloc[:,1] = temp.iloc[:,1] / total
neighborhoods_short = pd.concat([neighborhoods_short, temp['  Married or living common law']], axis = 1)

neighborhoods_short = pd.concat([neighborhoods_short,
                                 neighborhoods['Persons living alone (per cent)'].str.replace('%', '').astype(float)], axis = 1)

neighborhoods_short = pd.concat([neighborhoods_short,
                                 neighborhoods[' Average household size'].astype(float)], axis = 1)

neighborhoods_short = pd.concat([neighborhoods_short, 
                                 neighborhoods['Total income: Average amount ($)'].str.replace(',', '').astype(int)], axis = 1)


temp = pd.concat([neighborhoods['  Owner'].str.replace(',', '').astype(int),
                  neighborhoods['  Renter'].str.replace(',', '').astype(int)], axis = 1) 
total = temp.iloc[:,0] + temp.iloc[:,1]
temp.iloc[:,0] = temp.iloc[:,0] / total
temp.iloc[:,1] = temp.iloc[:,1] / total
neighborhoods_short = pd.concat([neighborhoods_short, temp['  Owner']], axis = 1)

temp = neighborhoods.iloc[:,[1702, 1703, 1705, 1708, 1709, 1711, 1712, 1713, 1714, 1715]].apply(pd.to_numeric)
total = temp.sum(axis = 1)
temp.iloc[:,0] = temp.iloc[:,0] / total
temp.iloc[:,1] = temp.iloc[:,1] / total
temp.iloc[:,2] = (temp.iloc[:,2] + temp.iloc[:,3] + temp.iloc[:,4]) / total
temp.iloc[:,5] = temp.iloc[:,5] / total
temp.iloc[:,6] = (temp.iloc[:,6] + temp.iloc[:,7] + temp.iloc[:,8] + temp.iloc[:,9]) / total

neighborhoods_short = pd.concat([neighborhoods_short, temp.iloc[:, [0,1,2,5,6]]], axis = 1)

neighborhoods_short = pd.concat([neighborhoods_short,
                                 neighborhoods['Participation rate'].astype(float),
                                 neighborhoods['Employment rate'].astype(float),
                                 neighborhoods['Unemployment rate'].astype(float)], axis = 1)

neighborhoods_short.head()

,LL,Neighborhood,Population density per square kilometre,Population Change 2011-2016,Children (0-14 years),Youth (15-24 years),Working Age (25-54 years),Pre-retirement (55-64 years),Seniors (65+ years),Older Seniors (85+ years),...,Total income: Average amount ($),Owner,"No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Apprenticeship or trades certificate or diploma,Bachelor's degree,University certificate or diploma above bachelor level,Participation rate,Employment rate,Unemployment rate
0,"43.807553, -79.268030",Agincourt North,3929,-3.9,0.127787,0.123295,0.376206,0.140765,0.201165,0.030782,...,30414,0.811300,0.261843,0.298221,0.207875,0.175095,0.056966,55.4,50.0,9.8
1,"43.604498, -79.541228",Alderwood,2435,1.3,0.142222,0.099798,0.421818,0.147475,0.162828,0.025859,...,47709,0.794595,0.195324,0.288358,0.293717,0.161715,0.060887,66.5,62.4,6.1
2,"43.671220, -79.404673",Annex,10863,4.6,0.074731,0.118746,0.476251,0.110196,0.187144,0.032932,...,112766,0.380414,0.060358,0.162605,0.146420,0.347867,0.282749,70.6,65.8,6.7
3,"43.765731, -79.457152",Bathurst Manor,3377,2.8,0.140060,0.116867,0.400904,0.122289,0.177108,0.042771,...,45936,0.539852,0.125424,0.255367,0.242561,0.231638,0.145009,65.0,60.3,7.2
4,"43.777287, -79.378287",Bayview Village,4195,21.1,0.109798,0.113890,0.468743,0.115481,0.164356,0.027734,...,52035,0.587834,0.069941,0.196476,0.211959,0.323278,0.198345,63.4,58.5,7.7


In [46]:
neighborhoods_desc = neighborhoods_short.describe()
neighborhoods_desc.head()

,Population density per square kilometre,Population Change 2011-2016,Children (0-14 years),Youth (15-24 years),Working Age (25-54 years),Pre-retirement (55-64 years),Seniors (65+ years),Older Seniors (85+ years),Married or living common law,Persons living alone (per cent),...,Total income: Average amount ($),Owner,"No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Apprenticeship or trades certificate or diploma,Bachelor's degree,University certificate or diploma above bachelor level,Participation rate,Employment rate,Unemployment rate
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,...,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,5540.696970,4.018182,0.154549,0.124560,0.420573,0.125790,0.152372,0.022156,0.509900,13.833333,...,43590.000000,0.545450,0.175833,0.265614,0.245029,0.203884,0.109640,62.778788,57.103030,9.142424
std,3416.597782,9.072295,0.034868,0.018163,0.057407,0.020630,0.041490,0.010923,0.049826,6.117938,...,19504.666895,0.185597,0.062134,0.041276,0.039360,0.067704,0.057438,5.716520,6.191652,2.148260
min,1260.000000,-4.600000,0.074731,0.086139,0.322180,0.077334,0.067188,0.004602,0.402399,3.900000,...,26793.000000,0.109628,0.060358,0.162605,0.146420,0.096358,0.043309,53.600000,48.600000,5.700000
25%,3148.000000,-0.500000,0.130435,0.113982,0.384464,0.113121,0.133025,0.014522,0.481662,10.500000,...,30974.000000,0.434419,0.128955,0.247530,0.210080,0.154203,0.067227,58.500000,51.900000,7.400000


 OLS Regression Results  
 
------------------------------------------------------------------------------

Dep. Variable:                  likes   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     15.71
Date:                Wed, 20 Mar 2019   Prob (F-statistic):           2.95e-06
Time:                        18:30:57   Log-Likelihood:                -97.964
No. Observations:                  33   AIC:                             203.9
Df Residuals:                      29   BIC:                             209.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         

                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   37.7569      8.311      4.543      0.000      20.759      54.755
Population density       0.0008      0.000      2.579      0.015       0.000       0.001
Postsecondary degree   -66.2331     28.721     -2.306      0.028    -124.974      -7.492
Unemployment rate       -1.4507      0.426     -3.408      0.002      -2.321      -0.580

==============================================================================

Omnibus:                        2.425   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.297   Jarque-Bera (JB):                1.298
Skew:                           0.148   Prob(JB):                        0.522
Kurtosis:                       3.926   Cond. No.                     2.20e+05

==============================================================================


### Predicting Venue Likes
We found that population density, postsecondary degree, and unemployment rates were the best predictors for venue likes.

### Predicting Venue Price
Price was difficult to predict with the available demographic data. The best we found was using average household size and unemployment rate, both of which are negatively correlated and account for only a third of the variability in price.

### Predicting Venue Rating
We see that percent of working age adults accounts for half of the variability of rating and that the higher percentage, the higher the rating will be. This suggests new businesses should open in areas with large working-age populations. The highest R-squared value we found, however, was for a model looking at unemployment rate, percent living alone, percentage of pre-retirement individuals, and percentage of people living alone. The lower the unemployment rate and the percert of pre-retirees, along with a slight increase in percent of people living alone, accounts for greater ratings.

### Predicting Demographic Information
In testing various regression models using Foursquare data to predict demographic information, we noticed that individual features can be fairly good predictors (accounding for 30 to 50% of targets), but combining predictors achieve no significant increase in R-squared. This is likely because the three Foursquare predictors are themselves highly correlated and don't contribute much to the model when added together.


## Conclusion

We began this project with the hope that we could find some linkages between Toronto's demographic data and Foursquare data for venues in Toronto. Although some of the results were promising, neither data sets gave an complete conclusions. We recommend that if both governments and business would like to use these results for their own predictions, that they do so carefully. The results show general trends, but cannot be used to predict exact amounts. There is still to much variability that is not explained by the predictors used here, both for predicting demographic information and Foursquare information. However, we do believe these results can be used for a quick check for new businesses to see if their other research matches with the results found in this project.